In [2]:
import json

# Archivos de entrada
input_files = [
    "tweets_griegos_stanza_1_500.json",
    "tweets_griegos_stanza_501_1000.json"
]
output_file = "tweets_españoles_evaluados_completo.json"

all_tweets = []

for path in input_files:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        print(f"Cargados {len(data)} tweets de {path}")
        # Asegurarse de que 'data' es una lista
        if isinstance(data, list):
            all_tweets.extend(data)
        else:
            raise ValueError(f"El archivo {path} no contiene una lista de tweets.")

print(f"Total combinado: {len(all_tweets)} tweets")

# Guardar resultado
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_tweets, f, ensure_ascii=False, indent=2)

print(f"✅ Todos los tweets guardados en {output_file}")


Cargados 500 tweets de tweets_griegos_stanza_1_500.json
Cargados 500 tweets de tweets_griegos_stanza_501_1000.json
Total combinado: 1000 tweets
✅ Todos los tweets guardados en tweets_españoles_evaluados_completo.json


In [5]:
import json
import csv

# 1) Rutas de entrada y salida
json_input  = "tweets_griegos_evaluados_completo.json"
csv_input   = "grecia_anotados/gr_combined.csv"                # CSV con columnas: id, label, annotator
json_output = "tweets_griegos_con_meta.json"

# 2) Leer registros del CSV
records = []
with open(csv_input, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        records.append(row)

# 3) Cargar el JSON combinado
with open(json_input, "r", encoding="utf-8") as f:
    tweets = json.load(f)

# 4) Verificar que coincidan en longitud
if len(records) != len(tweets):
    raise ValueError(f"El CSV tiene {len(records)} filas, pero el JSON tiene {len(tweets)} tweets.")

# 5) Asignar id, label y annotator
for rec, tweet in zip(records, tweets):
    tweet["id"]        = rec["id"]
    tweet["label"]     = rec.get("label")
    tweet["annotator"] = rec.get("annotator")

# 6) Guardar el JSON resultante
with open(json_output, "w", encoding="utf-8") as f:
    json.dump(tweets, f, ensure_ascii=False, indent=2)

print(f"✅ Metadatos añadidos a {len(tweets)} tweets. Archivo guardado en '{json_output}'.")


✅ Metadatos añadidos a 1000 tweets. Archivo guardado en 'tweets_griegos_con_meta.json'.


In [15]:
import json

INPUT_JSON = 'spain/Completado/tweets_españoles_evaluados_completo.json'

# Cargar archivo JSON
with open(INPUT_JSON, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Número de tweets
num_tweets = len(data)

# Número total de replies
total_replies = sum(len(item.get('replies', [])) for item in data)

# Media de replies por tweet
avg_replies = total_replies / num_tweets if num_tweets > 0 else 0

print(f"Número de tweets: {num_tweets}")
print(f"Número total de replies: {total_replies}")
print(f"Media de replies por tweet: {avg_replies:.2f}")


Número de tweets: 1000
Número total de replies: 3082
Media de replies por tweet: 3.08


In [26]:
import json
import unicodedata
from collections import defaultdict

# Rutas de entrada y salida
input_path = "spain/Completado/tweets_españoles_evaluados_completo.json"
output_path = "users_españoles.json"

# Función para normalizar nombres
def normalize_name(name):
    name = name.lower().strip()
    name = unicodedata.normalize('NFKD', name)
    name = ''.join(c for c in name if not unicodedata.combining(c))
    name = name.replace("(", "").replace(")", "")
    return " ".join(name.split())

# Carga de tweets
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Para evitar duplicados y asociar tweet IDs
users_dict = {}
seen_usernames = set()

# Para agrupar los tweet IDs por usuario original (raw_user)
user_to_ids = defaultdict(list)

for item in data:
    raw_user = item.get("user", "").strip()
    tweet_id = item.get("id")
    party = item.get("mp_party")

    if not raw_user or raw_user.startswith("Error al extraer usuario"):
        continue

    # Extraer nombre y @usuario
    if "\n" in raw_user:
        lines = [line.strip() for line in raw_user.splitlines() if line.strip()]
        if len(lines) >= 2 and lines[-1].startswith("@"):
            name = " ".join(lines[:-1])
            username = lines[-1]
        else:
            continue
    else:
        tokens = raw_user.split()
        username = None
        for t in reversed(tokens):
            if t.startswith("@"):
                username = t
                break
        if username:
            name = raw_user[: raw_user.rfind(username)].strip()
        else:
            continue

    # Agregar tweet ID
    if tweet_id:
        user_to_ids[username].append(tweet_id)

    # Si ya se procesó este username, solo agregamos el tweet
    if username in seen_usernames:
        continue

    seen_usernames.add(username)
    users_dict[username] = {
        "name": name,
        "user": username,
        "party": party,
        "normalized_name": normalize_name(name),
        "tweet_ids": []  # se añadirá más abajo
    }

# Añadir los IDs recolectados
for username, tweet_ids in user_to_ids.items():
    if username in users_dict:
        users_dict[username]["tweet_ids"] = tweet_ids

# Exportar a JSON
users = list(users_dict.values())
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(users, f, ensure_ascii=False, indent=2)

print(f"✅ Guardados {len(users)} usuarios únicos con tweets en '{output_path}'.")


✅ Guardados 172 usuarios únicos con tweets en 'users_españoles.json'.


In [2]:
import json
import re

# 1) Rutas de tus archivos
json_input  = "tweets_españoles_anotados_finales.json"     # tu JSON sin IDs
txt_input   = "tweets_id.txt"          # el archivo de log con "Procesando tweet ID: <id>"
json_output = "Tweets_españoles_id.json"     # salida con IDs asignados

# 2) Extraer la lista de IDs del txt en el orden en que aparecen
ids = []
with open(txt_input, "r", encoding="utf-8") as f:
    for line in f:
        m = re.search(r"Procesando tweet ID:\s*(\d+)", line)
        if m:
            ids.append(m.group(1))

# 3) Cargar tu JSON original
with open(json_input, "r", encoding="utf-8") as f:
    tweets = json.load(f)

# 4) Verificar que concuerden
if len(ids) != len(tweets):
    raise ValueError(f"Tengo {len(ids)} IDs en el txt pero {len(tweets)} tweets en el JSON.")

# 5) Asignar cada ID a su tweet correspondiente
for idx, tweet in enumerate(tweets):
    tweet["id"] = ids[idx]

# 6) Guardar el JSON resultante
with open(json_output, "w", encoding="utf-8") as f:
    json.dump(tweets, f, ensure_ascii=False, indent=2)

print(f"✅ Asignados {len(ids)} IDs. Salvo en '{json_output}'.")


✅ Asignados 1000 IDs. Salvo en 'Tweets_españoles_id.json'.


In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import unicodedata

# — Rutas de entrada/salida —
TWEETS_JSON    = 'spain/Completado/tweets_españoles_evaluados_completo.json'
DETAILS_JSON   = 'spain/politicos_españoles_con_edad.json'
USERS_JSON     = 'spain/users_españoles.json'
OUTPUT_JSON    = 'tweets_españoles_completos2.json'

# — Función para normalizar texto —
def normalize(text):
    text = text.lower().strip()
    text = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in text if not unicodedata.combining(c))

# — Carga de datos —
with open(TWEETS_JSON, encoding='utf-8') as f:
    all_tweets = json.load(f)

with open(DETAILS_JSON, encoding='utf-8') as f:
    politicos = json.load(f)

with open(USERS_JSON, encoding='utf-8') as f:
    user_info = json.load(f)

# — Mapeo rápido de ID → tweet —
id_to_tweet = {str(tw['id']): tw for tw in all_tweets}

# — Mapeo de nombre normalizado → @usuario —
name_to_user = {normalize(u['name']): u for u in user_info}

# — Construir resultado final —
final = {}
for pol in politicos:
    normname = normalize(pol['name'])
    user_entry = name_to_user.get(normname)

    if not user_entry:
        print(f"⚠️ No se encontró usuario para: {pol['name']}")
        continue

    tweet_ids = user_entry.get("tweet_ids", [])
    tweet_objs = []
    for tid in tweet_ids:
        t = id_to_tweet.get(str(tid))
        if t:
            tweet_objs.append({
                'id':         t.get('id'),
                'text':       t.get('text'),
                'valoration': t.get('valoration'),
                'replies':    t.get('replies', [])
            })
        else:
            print(f"❌ Tweet ID {tid} no encontrado en dataset.")

    final[normname] = {
        'name':   pol['name'],
        'user':   user_entry.get('user'),
        'age':    pol.get('age'),
        'party':  pol.get('party'),
        'gender': pol.get('gender'),
        'tweets': tweet_objs
    }

# — Guardar resultado —
output = list(final.values())
with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f"\n✅ JSON final guardado en: {OUTPUT_JSON}")


⚠️ No se encontró usuario para: Daniel Senderos
⚠️ No se encontró usuario para: María Jesus moro almaraz
⚠️ No se encontró usuario para: Víctor Sánchezdel Real
⚠️ No se encontró usuario para: Odón Elorza
⚠️ No se encontró usuario para: Joan Capdevila
⚠️ No se encontró usuario para: Juan Cuatrecasas Asúa
⚠️ No se encontró usuario para: Federico. J. Contreras
⚠️ No se encontró usuario para: Rocío De Meer
⚠️ No se encontró usuario para: Jose Luis Aceves
⚠️ No se encontró usuario para: María Luz Martínez Seijo
⚠️ No se encontró usuario para: José Ramírez del Río
⚠️ No se encontró usuario para: Jaume Alonso-Cuevillas
⚠️ No se encontró usuario para: Maria Dantas
⚠️ No se encontró usuario para: Concepción Cañadell
⚠️ No se encontró usuario para: Lídia Guinart Moreno
⚠️ No se encontró usuario para: Belén Fernández
⚠️ No se encontró usuario para: Joan Baldoví
⚠️ No se encontró usuario para: Pablo Iglesias
⚠️ No se encontró usuario para: Sònia Guerra López
⚠️ No se encontró usuario para: Bego Na

In [14]:
import json

# Ruta al archivo generado previamente
INPUT_JSON = 'spain/Completado/tweets_españoles_evaluados_completo.json'

with open(INPUT_JSON, encoding='utf-8') as f:
    data = json.load(f)

num_users = len(data)
total_tweets = 0
total_replies = 0
users_sin_tweets = []

for entry in data:
    tweets = entry.get('tweets', [])
    total_tweets += len(tweets)

    if not tweets:
        users_sin_tweets.append(entry['user'])

    for tweet in tweets:
        replies = tweet.get('replies', [])
        total_replies += len(replies)

# Cálculo de media de replies por tweet
avg_replies = total_replies / total_tweets if total_tweets > 0 else 0

# Resultados
print("📊 Estadísticas generales")
print(f"👤 Número de usuarios:       {num_users}")
print(f"💬 Total de tweets:          {total_tweets}")
print(f"💭 Total de respuestas:      {total_replies}")
print(f"📈 Promedio respuestas/tweet:{avg_replies:.2f}")

# Usuarios sin tweets
if users_sin_tweets:
    print("\n⚠️ Usuarios sin tweets:")
    for name in users_sin_tweets:
        print(f"  - {name}")
else:
    print("\n✅ Todos los usuarios tienen al menos un tweet.")


📊 Estadísticas generales
👤 Número de usuarios:       1000
💬 Total de tweets:          0
💭 Total de respuestas:      0
📈 Promedio respuestas/tweet:0.00

⚠️ Usuarios sin tweets:
  - Juan López de Uralde
@juralde
·
Nov 7, 2021
  - Gloria Elizo
@GloriaElizo
  - Guillermo Prudencio
@guilleprudencio
·
Feb 19, 2021
  - Andrés Lorite
@AndresLorite
  - Inés Sabanés
@isabanes
  - José Alcaraz Martos
@fjosealcaraz
  - Marisa Saavedra
@MarisaSaavedraM
  - Daniel Senderos/
@danielsenderos
  - Jaume Asens
@Jaumeasens
  - Pilar Marcos
@pilarmarcosd
·
Sep 9, 2021
  - Marta Martín Llaguno
@martamartirio
  - Marisa Saavedra
@MarisaSaavedraM
  - Marisa Saavedra
@MarisaSaavedraM
  - Error al extraer usuario de 1356204051105460227 tras 3 intentos.
  - Inès Gumà Mitjà
@inheis
·
Aug 28, 2021
  - Diego Gago Bugarín
@DiegoGagoB
  - Error al extraer usuario de 1444931213400199170 tras 3 intentos.
  - Miriam Nogueras
@miriamnoguerasM
·
Sep 23, 2021
  - Miguel Gutiérrez
@MGutierrezCs
  - Rosa Romero
@rosaromerocr

In [30]:
import json
import unicodedata

# — Rutas de entrada y salida —
POLITICOS_PATH = "spain/politicos_españoles_con_edad.json"
TWEETS_PATH = "tweets_españoles_evaluados_completo.json"
OUTPUT_PATH = "politicos_con_tweets_completos.json"

# — Función de normalización —
def normalize(text):
    text = text.lower().strip()
    text = unicodedata.normalize("NFKD", text)
    return ''.join(c for c in text if not unicodedata.combining(c))

# — Cargar archivos —
with open(POLITICOS_PATH, "r", encoding="utf-8") as f:
    politicos = json.load(f)

with open(TWEETS_PATH, "r", encoding="utf-8") as f:
    tweets_data = json.load(f)

# — Agrupar tweets por @usuario —
def construir_tweet(tweet):
    return {
        "id": tweet.get("id"),
        "text": tweet.get("tweet"),
        "valoration": tweet.get("analysis", {}),
        "replies": tweet.get("replies", [])
    }

tweets_por_user = {}
for tw in tweets_data:
    raw_user = tw.get("user", "")
    if not raw_user or raw_user.startswith("Error"):
        continue

    # Extraer el @handle
    tokens = raw_user.split()
    handle = next((t for t in reversed(tokens) if t.startswith("@")), None)
    if not handle:
        continue

    if handle not in tweets_por_user:
        tweets_por_user[handle] = []

    tweets_por_user[handle].append(construir_tweet(tw))

# — Fusionar políticos con tweets —
with open("users_españoles.json", "r", encoding="utf-8") as f:
    user_map = json.load(f)

# Mapeo rápido: @usuario → nombre normalizado
handle_to_normname = {u['user']: normalize(u['name']) for u in user_map}

# Generar resultado
final = []
for p in politicos:
    normname = normalize(p['name'])
    handle = next((h for h, n in handle_to_normname.items() if n == normname), None)

    tweets = tweets_por_user.get(handle, []) if handle else []

    final.append({
        "name": p["name"],
        "user": handle,
        "party": p.get("party"),
        "age": p.get("age"),
        "gender": p.get("gender"),
        "tweets": tweets
    })

# — Guardar JSON final —
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(final, f, ensure_ascii=False, indent=2)

print(f"✅ JSON final generado en: {OUTPUT_PATH}")


FileNotFoundError: [Errno 2] No such file or directory: 'tweets_españoles_evaluados_completo.json'

In [37]:
import pandas as pd

# Ruta del archivo Excel de entrada
input_excel = "grecia_anotados/politicos_griegos.xlsx"  # Cambia por tu archivo

# Leer la hoja (puedes especificar sheet_name si hay varias hojas)
df = pd.read_excel(input_excel)

# Convertir a JSON y guardar
output_json = "grecia_anotados/politicos_griegos.json"
df.to_json(output_json, orient="records", force_ascii=False, indent=2)

print(f"Archivo guardado como: {output_json}")

Archivo guardado como: grecia_anotados/politicos_griegos.json


In [26]:
import pandas as pd
import json
import stanza

# Paso 1: Descargar modelos si es necesario
stanza.download("es")

# Paso 2: Inicializar el pipeline de Stanza para español
nlp = stanza.Pipeline("es", processors="tokenize,pos", use_gpu=False)

# Paso 3: Cargar archivo Excel
archivo_excel = "spain/Tweets_etiquetados.xlsx"
df = pd.read_excel(archivo_excel)

# Paso 4: Procesar cada tweet
resultados = []

for _, row in df.iterrows():
    tweet_text = str(row["text"])
    user = str(row["id"]) if "id" in row else "desconocido"

    # POS tagging con Stanza
    doc = nlp(tweet_text)
    pos_tags = []
    counts = {"DET": 0, "NOUN": 0, "VERB": 0, "ADJ": 0, "PROPN": 0, "PRON": 0}

    first_word_pos = None

    for sentence in doc.sentences:
        for idx, word in enumerate(sentence.words):
            pos_tags.append({
                "text": word.text,
                "pos": word.upos
            })
            if word.upos in counts:
                counts[word.upos] += 1
            if first_word_pos is None and idx == 0:
                first_word_pos = word.upos

    # Paso 5: Construir el diccionario de salida SIN buscar partidos
    resultados.append({
        "id": user,
        "tweet": tweet_text,
        "num_determinantes": counts["DET"],
        "num_nouns": counts["NOUN"],
        "num_verbs": counts["VERB"],
        "num_adjectives": counts["ADJ"],
        "num_proper_nouns": counts["PROPN"],
        "starts_with_pronoun": first_word_pos == "PRON",
        "ratio_noun_to_verb": round(counts["NOUN"] / counts["VERB"], 2) if counts["VERB"] > 0 else None
    })

# Paso 6: Guardar el resultado en JSON
output_file = "tweets_features_stanza.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=2)

print(f"✅ Archivo enriquecido guardado como: {output_file}")


2025-04-29 21:23:56 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-04-29 21:23:56 INFO: Downloading default packages for language: es (Spanish) ...
2025-04-29 21:23:57 INFO: File exists: /home/jupyter-lquijano/stanza_resources/es/default.zip
2025-04-29 21:24:01 INFO: Finished downloading models and saved to /home/jupyter-lquijano/stanza_resources
2025-04-29 21:24:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-29 21:24:01 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-04-29 21:24:01 WARNING: Language es package default expects mwt, which has been added
2025-04-29 21:24:01 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2025-04-29 21:24:01 INFO: Using device: cpu
2025-04-29 21:24:01 INFO: Loading: tokenize
2025-04-29 21:24:01 INFO: Loading: mwt
2025-04-29 21:24:01 INFO: Loading: pos
2025-04-29 21:24:03 INFO: Done loading processors!


✅ Archivo enriquecido guardado como: tweets_features_stanza.json


In [31]:
import json
import stanza

# Paso 1: Descargar modelo de griego si es necesario
stanza.download("el")

# Paso 2: Inicializar el pipeline de Stanza para griego
nlp = stanza.Pipeline("el", processors="tokenize,pos", use_gpu=True)

# Paso 3: Cargar archivo JSON
archivo_json = "grecia_anotados/Preprado/Tweets_griegos.json"
with open(archivo_json, "r", encoding="utf-8") as f:
    data = json.load(f)

# Paso 4: Procesar cada tweet
resultados = []

for item in data:
    tweet_text = str(item.get("tweet", ""))
    tweet_id = str(item.get("id", "desconocido"))

    # POS tagging con Stanza
    doc = nlp(tweet_text)
    pos_tags = []
    counts = {"DET": 0, "NOUN": 0, "VERB": 0, "ADJ": 0, "PROPN": 0, "PRON": 0}
    first_word_pos = None

    for sentence in doc.sentences:
        for idx, word in enumerate(sentence.words):
            pos_tags.append({
                "text": word.text,
                "pos": word.upos
            })
            if word.upos in counts:
                counts[word.upos] += 1
            if first_word_pos is None and idx == 0:
                first_word_pos = word.upos

    # Paso 5: Añadir los features directamente al objeto original
    item["pos_features"] = {
        "num_determinantes": counts["DET"],
        "num_nouns": counts["NOUN"],
        "num_verbs": counts["VERB"],
        "num_adjectives": counts["ADJ"],
        "num_proper_nouns": counts["PROPN"],
        "starts_with_pronoun": first_word_pos == "PRON",
        "ratio_noun_to_verb": round(counts["NOUN"] / counts["VERB"], 2) if counts["VERB"] > 0 else None
    }

# Paso 6: Guardar el resultado en un nuevo JSON
output_file = "tweets_griegos_features_stanza.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ Archivo enriquecido guardado como: {output_file}")


2025-04-29 21:48:42 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-04-29 21:48:42 INFO: Downloading default packages for language: el (Greek) ...
2025-04-29 21:48:43 INFO: File exists: /home/jupyter-lquijano/stanza_resources/el/default.zip
2025-04-29 21:48:44 INFO: Finished downloading models and saved to /home/jupyter-lquijano/stanza_resources
2025-04-29 21:48:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-29 21:48:44 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-04-29 21:48:44 WARNING: Language el package default expects mwt, which has been added
2025-04-29 21:48:44 INFO: Loading these models for language: el (Greek):
| Processor | Package      |
----------------------------
| tokenize  | gdt          |
| mwt       | gdt          |
| pos       | gdt_nocharlm |

2025-04-29 21:48:44 WARNING: GPU requested, but is not available!
2025-04-29 21:48:44 INFO: Using device: cpu
2025-04-29 21:48:44 INFO: Loading: tokenize
2025-04-29 21:48:44 INFO: Loading: mwt
2025-04-29 21:48:44 INFO: Loading: pos
2025-04-29 21:48:45 INFO: Done loading processors!


✅ Archivo enriquecido guardado como: tweets_griegos_features_stanza.json


In [34]:
import json

# 1. Cargar el JSON principal (con análisis de sentimiento, emociones, etc.)
with open("grecia_anotados/Preprado/Tweets_griegos.json", "r", encoding="utf-8") as f:
    tweets_analizados = json.load(f)

# 2. Cargar el JSON con los features POS (stanza)
with open("tweets_griegos_features_stanza.json", "r", encoding="utf-8") as f:
    tweets_features = json.load(f)

# 3. Crear un mapa rápido: texto del tweet -> pos_features
mapa_features = {
    item["tweet"].strip(): item.get("pos_features", {})
    for item in tweets_features
}

# 4. Insertar los pos_features en el analysis de cada tweet
for tweet in tweets_analizados:
    texto = tweet["tweet"].strip()
    features = mapa_features.get(texto, None)
    if features:
        tweet.setdefault("analysis", {})["pos_features"] = features  # Añadir pos_features dentro de analysis

# 5. Guardar el nuevo JSON enriquecido
with open("tweets_completos_con_analysis_y_pos.json", "w", encoding="utf-8") as f:
    json.dump(tweets_analizados, f, ensure_ascii=False, indent=2)

print("✅ Se añadieron los POS features al analysis de cada tweet correctamente.")


✅ Se añadieron los POS features al analysis de cada tweet correctamente.


In [7]:
import json

# 1. Cargar JSON con los análisis y partidos
with open("grecia_anotados/Preprado/Griegos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. Crear diccionario de partidos → polaridad
partido_a_ideologia = {
    "Νέα Δημοκρατία": "derecha",
    "Ελληνική Λύση": "derecha",
    "Ανεξάρτητοι Έλληνες": "derecha",
    "Εθνική Δημιουργία": "derecha",
    "Συνασπισμός Ριζοσπαστικής Αριστεράς – Προοδευτική Συμμαχία": "izquierda",
    "ΠΑΣΟΚ – Κίνημα Αλλαγής": "izquierda",
    "ΜέΡΑ25": "izquierda",
    "Κίνημα Αλλαγής": "izquierda",
    "Κομμουνιστικό Κόμμα Ελλάδας": "izquierda",
    "Κίνημα Δημοκρατών Σοσιαλιστών": "izquierda"
}

# 3. Suponemos que en cada tweet tienes la clave 'party' ya puesta
for tweet in data:
    partido = tweet.get("party", "").strip()
    polaridad = partido_a_ideologia.get(partido)
    tweet["polaridad_ideologica"] = polaridad  # Puede ser None si el partido no está en el mapeo

# 4. Guardar resultado
with open("grecia_anotados/Preprado/Griegos_f.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Archivo actualizado con la polaridad ideológica.")


✅ Archivo actualizado con la polaridad ideológica.


In [44]:
import json

# Carga tus datos (aquí se asume que ya los has cargado como listas de diccionarios)
# json1 es la lista con tweets, json2 la lista con información personal

with open('grecia_anotados/Preprado/tweets_completos_con_analysis_y_pos.json', 'r', encoding='utf-8') as f:
    tweets_data = json.load(f)

with open('grecia_anotados/politicos_griegos.json', 'r', encoding='utf-8') as f:
    info_data = json.load(f)

# Crear un diccionario para búsqueda rápida por nombre
info_dict = {entry['name']: entry for entry in info_data}

# Función para extraer el nombre del campo "user"
def extract_name(user_str):
    return user_str.split('\n')[0].strip()

# Lista final combinada
combined = []

for tweet_entry in tweets_data:
    name = extract_name(tweet_entry["user"])
    persona_info = info_dict.get(name, {})  # Si no se encuentra, será un dict vacío
    
    tweet_copy = tweet_entry.copy()
    tweet_copy["name"] = name  # Añadir campo separado
    tweet_copy["age"] = persona_info.get("age")
    tweet_copy["gender"] = persona_info.get("gender")
    tweet_copy["party"] = persona_info.get("party")
    
    combined.append(tweet_copy)

# Guardar resultado
with open('combined.json', 'w', encoding='utf-8') as f:
    json.dump(combined, f, ensure_ascii=False, indent=2)

# Mostrar una muestra
combined[:2]


[{'user': 'Giannis Plakiotakis\n@G_Plakiotakis',
  'tweet': 'Την Τετάρτη το πρωί, στις 9:15 στον ΣΚΑΪ και την εκπομπή "Σήμερα" #Πλακιωτάκης #ΣΚΑΪ https://t.co/PynuyD0upJ',
  'replies': [],
  'analysis': {'sentiment': {'label': 'neutral', 'score': 0.8632032871246338},
   'emotions': [{'label': 'optimism', 'score': 0.593174934387207},
    {'label': 'joy', 'score': 0.19987452030181885},
    {'label': 'anger', 'score': 0.12021705508232117}],
   'hate': {'label': 'not hate', 'score': 0.7837180495262146},
   'translation': 'Wednesday morning, at 9:15 p.m. and today\'s "Today" issue #Scaoits https://t.co/PynuyD0upJ',
   'summary': ' Wednesday morning, at 9:15 p.m. and today\'s "Today" issue #Scaoits .',
   'pos_features': {'num_determinantes': 5,
    'num_nouns': 6,
    'num_verbs': 0,
    'num_adjectives': 0,
    'num_proper_nouns': 0,
    'starts_with_pronoun': False,
    'ratio_noun_to_verb': None}},
  'id': '1349005840745984007',
  'label': '0',
  'annotator': 'tiebreak_rule',
  'name': '

In [61]:
import json
import unicodedata

# Función para normalizar nombres (quita acentos, pone minúsculas)
def normalizar(texto):
    if not texto:
        return ""
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    return texto.lower().strip()

# Cargar el mapeo nombre–usuario
with open('grecia_anotados/users_griegos.json', 'r', encoding='utf-8') as f:
    name_user_map = json.load(f)

# Cargar la información personal original
with open('grecia_anotados/politicos_griegos.json', 'r', encoding='utf-8') as f:
    info_data = json.load(f)

# Crear diccionario con nombres normalizados para búsqueda flexible
info_dict_normalizado = {
    normalizar(entry["name"]): entry
    for entry in info_data
}

# Construir el nuevo JSON unificado
resultado = []
no_encontrados = []

for entry in name_user_map:
    name_original = entry["name"]
    user = entry["user"]
    name_normalizado = normalizar(name_original)

    info = info_dict_normalizado.get(name_normalizado)

    if info:
        resultado.append({
            "name": name_original,
            "user": user,
            "age": info.get("age"),
            "gender": info.get("gender"),
            "party": info.get("party")
        })
    else:
        no_encontrados.append(name_original)
        resultado.append({
            "name": name_original,
            "user": user,
            "age": None,
            "gender": None,
            "party": None
        })

# Guardar resultado
with open('user_info_completo.json', 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)

# Guardar lista de no encontrados (opcional)
with open('no_encontrados.json', 'w', encoding='utf-8') as f:
    json.dump(no_encontrados, f, ensure_ascii=False, indent=2)
print(len(resultado))
# Mostrar cuántos no se encontraron
print(f"No se encontró información para {len(no_encontrados)} personas.")


142
No se encontró información para 73 personas.


In [3]:
import json
from collections import defaultdict

# Cargar archivos
with open('spain/Completado/tweets_españoles_evaluados_completo.json', 'r', encoding='utf-8') as f:
    original_data = json.load(f)

with open('spain/Tweets_etiquetados.json', 'r', encoding='utf-8') as f:
    manual_annotations = json.load(f)

with open('tweets_españoles_evaluados_completo.json', 'r', encoding='utf-8') as f:
    stanza_sentiment = json.load(f)

with open('spain/usuarios_con_tweets_y_detalles.json', 'r', encoding='utf-8') as f:
    user_info = json.load(f)

# Convertir anotaciones y sentimientos en diccionarios para acceso rápido
annotation_dict = {str(item['id']): item for item in manual_annotations}
stanza_dict = {str(item['id']): item['analysis']['sentiment'] for item in stanza_sentiment}
stanza_reply_dict = {str(item['id']): item for item in stanza_sentiment}

# Crear índice de usuario por tweet_id
user_by_tweet = {}
for user in user_info:
    for tid in user['tweet_ids']:
        user_by_tweet[str(tid)] = {
            "name": user['name'],
            "age": user['age'],
            "gender": user['gender'],
            "party": user['party']
        }

# Organizar resultado final por usuario
final_result = defaultdict(lambda: {
    "name": "",
    "age": None,
    "gender": "",
    "party": "",
    "tweets": []
})

for tweet in original_data:
    tweet_id = str(tweet['id'])

    # Filtrar si no está en las anotaciones
    if tweet_id not in annotation_dict:
        continue

    # Verificar si hay datos del usuario
    if tweet_id not in user_by_tweet:
        continue

    user_info_obj = user_by_tweet[tweet_id]
    user_key = user_info_obj['name'] + ' ' + user_info_obj['party']

    # Agregar información del usuario
    final_result[user_key]['name'] = user_info_obj['name']
    final_result[user_key]['age'] = user_info_obj['age']
    final_result[user_key]['gender'] = user_info_obj['gender']
    final_result[user_key]['party'] = user_info_obj['party']

    # Agregar anotaciones manuales
    annotations = {
        "User1": annotation_dict[tweet_id]["User1"],
        "User2": annotation_dict[tweet_id]["User2"],
        "Total": annotation_dict[tweet_id]["Total"]
    }

    # Añadir sentimiento stanza si existe
    if tweet_id in stanza_dict:
        tweet['analysis']['sentiment_stanza'] = stanza_dict[tweet_id]

    # Procesar replies por orden
    replies = tweet.get('replies', [])
    stanza_replies = stanza_reply_dict.get(tweet_id, {}).get('replies', [])

    for idx, reply in enumerate(replies):
        if idx < len(stanza_replies):
            sentiment_stanza = stanza_replies[idx].get('analysis', {}).get('sentiment')
            if sentiment_stanza is not None:
                reply['analysis']['sentiment_stanza'] = sentiment_stanza

    # Crear estructura final del tweet
    tweet_struct = {
        "id": tweet_id,
        "tweet": tweet['tweet'],
        "analysis": tweet['analysis'],
        "annotations": annotations,
        "replies": replies
    }

    final_result[user_key]['tweets'].append(tweet_struct)

# Guardar resultado final
final_json = list(final_result.values())

with open('tweets_fusionados.json', 'w', encoding='utf-8') as f:
    json.dump(final_json, f, indent=2, ensure_ascii=False)

print("✅ Fusión completada. Archivo guardado como 'tweets_fusionados.json'.")


✅ Fusión completada. Archivo guardado como 'tweets_fusionados.json'.


In [6]:
import json

# Cargar el archivo fusionado
with open('tweets_fusionados_ideologia.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extraer todos los partidos
partidos = set()
for usuario in data:
    partido = usuario.get('party')
    if partido:
        partidos.add(partido)

# Mostrar la lista ordenada
partidos_ordenados = sorted(partidos)
print("Partidos políticos encontrados:")
for partido in partidos_ordenados:
    print("-", partido)


Partidos políticos encontrados:
- BNG
- CCa-PNC-NC
- CUP-PR
- Cs
- ECP
- EH Bildu
- ERC-Sob
- JxCat
- MP
- NA+
- PP
- PSOE
- UP
- VOX


In [5]:
import json

# Cargar el archivo fusionado
with open('tweets_fusionados.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Normalizar nombres de partidos (guiones inconsistentes)
def normalizar_partido(partido):
    partido = partido.replace('–', '-')  # guion largo a guion corto
    if partido in ['ERC-Sob', 'ERC–Sob']:
        return 'ERC-Sob'
    return partido

# Definir agrupación ideológica
izquierda = {'BNG', 'CUP-PR', 'ECP', 'EH Bildu', 'ERC-Sob', 'MP', 'PSOE', 'UP'}
derecha = {'CCa-PNC-NC', 'Cs', 'JxCat', 'NA+', 'PP', 'VOX'}

# Procesar cada usuario
for usuario in data:
    partido = normalizar_partido(usuario['party'])
    usuario['party'] = partido  # actualizar nombre normalizado

    if partido in izquierda:
        usuario['ideologia'] = 'izquierda'
    elif partido in derecha:
        usuario['ideologia'] = 'derecha'
    else:
        usuario['ideologia'] = 'otros'

# Guardar el archivo modificado
with open('tweets_fusionados_ideologia.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Partidos normalizados y clasificados. Guardado como 'tweets_fusionados_ideologia.json'.")


✅ Partidos normalizados y clasificados. Guardado como 'tweets_fusionados_ideologia.json'.


In [2]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === 1. Cargar archivo fusionado con ideología incluida ===
with open('tweets_fusionados_ideologia.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer replies con análisis y atributos del autor original ===
rows = []
for usuario in data:
    gender = usuario['gender']
    age = usuario['age']
    party = usuario['party']
    ideologia = usuario['ideologia']

    for tweet in usuario['tweets']:
        for reply in tweet.get('replies', []):
            if not isinstance(reply, dict):
                continue
            analysis = reply.get('analysis')
            if not isinstance(analysis, dict):
                continue

            hate = analysis.get('hate')
            hate_score = hate.get('score') if isinstance(hate, dict) else None

            sentiment = analysis.get('sentiment')
            sentiment_score = sentiment.get('score') if isinstance(sentiment, dict) else None

            emotions_raw = analysis.get('emotions', [])
            emotions = {}
            if isinstance(emotions_raw, list):
                emotions = {
                    e['label']: e['score']
                    for e in emotions_raw
                    if isinstance(e, dict) and 'label' in e and 'score' in e
                }

            row = {
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'hate_score': hate_score,
                'sentiment_score': sentiment_score,
            }
            row.update(emotions)
            rows.append(row)

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Limpiar y categorizar edad ===
df = df.dropna(subset=['age'])  # eliminar edad nula
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Función para calcular d de Cohen ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled

# === 6. Comparar todas las métricas contra todos los grupos ===
metricas = ['hate_score', 'sentiment_score', 'anger', 'joy', 'sadness', 'optimism']
grupos = ['gender', 'ideologia', 'age_group']

resultados = []

for variable_grupo in grupos:
    for metrica in metricas:
        grupos_unicos = df[variable_grupo].dropna().unique()
        if len(grupos_unicos) != 2:
            continue

        grupo1, grupo2 = grupos_unicos
        datos1 = df[df[variable_grupo] == grupo1][metrica].dropna()
        datos2 = df[df[variable_grupo] == grupo2][metrica].dropna()

        if len(datos1) < 10 or len(datos2) < 10:
            continue

        d = cohens_d(datos1, datos2)
        t_stat, p_value = ttest_ind(datos1, datos2, equal_var=False)

        resultados.append({
            "grupo": variable_grupo,
            "valor": metrica,
            "grupo_1": grupo1,
            "grupo_2": grupo2,
            "d_cohen": d,
            "t_stat": t_stat,
            "p_value": p_value,
            "n1": len(datos1),
            "n2": len(datos2)
        })

# === 7. Mostrar resultados ordenados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)

# === 8. Mostrar como tabla interactiva en Jupyter ===
df_resultados.to_csv("comparaciones_sesgo.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'comparaciones_sesgo.csv'")



✅ Resultados guardados en 'comparaciones_sesgo.csv'


In [11]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
# === Función para calcular el tamaño del efecto (Cohen's d) ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0.0

# === 1. Cargar archivo JSON ===
with open('grecia_anotados/Preprado/Griegos_f.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer predicciones y etiquetas humanas ===
rows = []
for usuario in data:
    gender = usuario.get('gender')
    age = usuario.get('age')
    party = usuario.get('party')
    ideologia = usuario.get('polaridad_ideologica')

    for tweet in usuario.get('tweets', []):
        for reply in [tweet] + tweet.get('replies', []):
            if not isinstance(reply, dict):
                continue

            label_humana = reply.get('label')
            if label_humana not in ['-1', '0', '1']:
                continue

            analysis = reply.get('analysis')
            if not isinstance(analysis, dict):
                continue

            pred_sentiment = analysis.get('sentiment', {}).get('label')
            if pred_sentiment not in ['positive', 'neutral', 'negative']:
                continue

            # Convertimos para comparar: humano (str) -> modelo (str)
            mapping_humano = {'-1': 'negative', '0': 'neutral', '1': 'positive'}
            label_humana_str = mapping_humano.get(label_humana)

            acierto = int(pred_sentiment == label_humana_str)

            rows.append({
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'acierto': acierto
            })

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Comparar tasa de acierto por grupo ===
grupos = ['gender', 'ideologia', 'age_group', 'party']
resultados = []

for variable in grupos:
    grupos_unicos = df[variable].dropna().unique()
    if len(grupos_unicos) != 2:
        continue

    g1, g2 = grupos_unicos
    acc1 = df[df[variable] == g1]['acierto']
    acc2 = df[df[variable] == g2]['acierto']

    if len(acc1) < 10 or len(acc2) < 10:
        continue

    # Diferencia en tasa de acierto
    mean1, mean2 = acc1.mean(), acc2.mean()
    d = cohens_d(acc1, acc2)
    t_stat, p_value = ttest_ind(acc1, acc2, equal_var=False)

    resultados.append({
        'grupo': variable,
        'grupo_1': g1,
        'grupo_2': g2,
        'acierto_1': mean1,
        'acierto_2': mean2,
        'diff': mean1 - mean2,
        'd_cohen': d,
        't_stat': t_stat,
        'p_value': p_value,
        'n1': len(acc1),
        'n2': len(acc2)
    })

# === 6. Resultados ordenados por magnitud de efecto ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)

# === 7. Guardar como CSV ===
df_resultados.to_csv("sesgo_modelo_sentimiento.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'sesgo_modelo_sentimiento.csv'")


✅ Resultados guardados en 'sesgo_modelo_sentimiento.csv'


In [10]:
print(df['ideologia'].value_counts(dropna=False))


ideologia
None    925
Name: count, dtype: int64


In [1]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === 1. Cargar archivo con estructura nueva ===
with open('grecia_anotados/Preprado/Griegos.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer tweets originales y replies con análisis + atributos del autor ===
rows = []
for persona in data:
    gender = persona.get('gender')
    age = persona.get('age')
    party = persona.get('party')
    ideologia = persona.get('ideologia')

    for tweet in persona.get('tweets', []):
        for fuente in ['analysis'] + [r.get('analysis') for r in tweet.get('replies', []) if isinstance(r, dict) and 'analysis' in r]:
            if not isinstance(fuente, dict):
                continue

            hate = fuente.get('hate')
            hate_score = hate.get('score') if isinstance(hate, dict) else None

            sentiment = fuente.get('sentiment')
            sentiment_score = sentiment.get('score') if isinstance(sentiment, dict) else None

            emotions_raw = fuente.get('emotions', [])
            emotions = {}
            if isinstance(emotions_raw, list):
                emotions = {
                    e['label']: e['score']
                    for e in emotions_raw
                    if isinstance(e, dict) and 'label' in e and 'score' in e
                }

            row = {
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'hate_score': hate_score,
                'sentiment_score': sentiment_score,
            }
            row.update(emotions)
            rows.append(row)

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Limpiar y categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Función para calcular d de Cohen ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled

# === 6. Comparar métricas entre grupos ===
metricas = ['hate_score', 'sentiment_score', 'anger', 'joy', 'sadness', 'optimism']
grupos = ['gender', 'ideologia', 'age_group']

resultados = []

for variable_grupo in grupos:
    for metrica in metricas:
        grupos_unicos = df[variable_grupo].dropna().unique()
        if len(grupos_unicos) != 2:
            continue

        grupo1, grupo2 = grupos_unicos
        datos1 = df[df[variable_grupo] == grupo1][metrica].dropna()
        datos2 = df[df[variable_grupo] == grupo2][metrica].dropna()

        if len(datos1) < 10 or len(datos2) < 10:
            continue

        d = cohens_d(datos1, datos2)
        t_stat, p_value = ttest_ind(datos1, datos2, equal_var=False)

        resultados.append({
            "grupo": variable_grupo,
            "valor": metrica,
            "grupo_1": grupo1,
            "grupo_2": grupo2,
            "d_cohen": d,
            "t_stat": t_stat,
            "p_value": p_value,
            "n1": len(datos1),
            "n2": len(datos2)
        })

# === 7. Guardar resultados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)
df_resultados.to_csv("comparaciones_completas.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'comparaciones_completas.csv'")


✅ Resultados guardados en 'comparaciones_completas.csv'


In [19]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === 1. Cargar archivo JSON ===
with open('spain/Completado/Spain_Completo.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer datos de análisis y atributos del autor ===
rows = []
for persona in data:
    gender = persona.get('gender')
    age = persona.get('age')
    party = persona.get('party')
    ideologia = persona.get('ideologia')

    for tweet in persona.get('tweets', []):
        # Procesar tweet original
        fuentes = [tweet.get('analysis')]
        # Añadir replies
        fuentes += [r.get('analysis') for r in tweet.get('replies', []) if isinstance(r, dict) and 'analysis' in r]

        for fuente in fuentes:
            if not isinstance(fuente, dict):
                continue

            hate = fuente.get('hate')
            hate_score = hate.get('score') if isinstance(hate, dict) else None

            sentiment = fuente.get('sentiment')
            sentiment_score = sentiment.get('score') if isinstance(sentiment, dict) else None

            emotions_raw = fuente.get('emotions', [])
            emotions = {}
            if isinstance(emotions_raw, list):
                emotions = {
                    e['label']: e['score']
                    for e in emotions_raw
                    if isinstance(e, dict) and 'label' in e and 'score' in e
                }

            row = {
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'hate_score': hate_score,
                'sentiment_score': sentiment_score,
            }
            row.update(emotions)
            rows.append(row)

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Limpiar y categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Función para calcular d de Cohen ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0.0

# === 6. Comparar métricas entre grupos ===
metricas = ['hate_score', 'sentiment_score', 'anger', 'joy', 'sadness', 'optimism']
grupos = ['gender', 'ideologia', 'age_group']

resultados = []

for variable_grupo in grupos:
    for metrica in metricas:
        grupos_unicos = df[variable_grupo].dropna().unique()
        if len(grupos_unicos) != 2:
            continue

        grupo1, grupo2 = grupos_unicos
        datos1 = df[df[variable_grupo] == grupo1][metrica].dropna()
        datos2 = df[df[variable_grupo] == grupo2][metrica].dropna()

        if len(datos1) < 10 or len(datos2) < 10:
            continue

        d = cohens_d(datos1, datos2)
        t_stat, p_value = ttest_ind(datos1, datos2, equal_var=False)

        resultados.append({
            "grupo": variable_grupo,
            "valor": metrica,
            "grupo_1": grupo1,
            "grupo_2": grupo2,
            "d_cohen": d,
            "t_stat": t_stat,
            "p_value": p_value,
            "n1": len(datos1),
            "n2": len(datos2)
        })

# === 7. Guardar resultados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)
df_resultados.to_csv("comparaciones_metricas_espanoles.csv", index=False, encoding='utf-8')

print("✅ Resultados guardados en 'comparaciones_metricas_espanoles.csv'")


✅ Resultados guardados en 'comparaciones_metricas_espanoles.csv'


In [ ]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
# === Función para calcular el tamaño del efecto (Cohen's d) ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0.0

# === 1. Cargar archivo JSON ===
with open('grecia_anotados/Preprado/Griegos_f.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer predicciones y etiquetas humanas ===
rows = []
for usuario in data:
    gender = usuario.get('gender')
    age = usuario.get('age')
    party = usuario.get('party')
    ideologia = usuario.get('polaridad_ideologica')

    for tweet in usuario.get('tweets', []):
        for reply in [tweet] + tweet.get('replies', []):
            if not isinstance(reply, dict):
                continue

            label_humana = reply.get('label')
            if label_humana not in ['-1', '0', '1']:
                continue

            analysis = reply.get('analysis')
            if not isinstance(analysis, dict):
                continue

            pred_sentiment = analysis.get('sentiment', {}).get('label')
            if pred_sentiment not in ['positive', 'neutral', 'negative']:
                continue

            # Convertimos para comparar: humano (str) -> modelo (str)
            mapping_humano = {'-1': 'negative', '0': 'neutral', '1': 'positive'}
            label_humana_str = mapping_humano.get(label_humana)

            acierto = int(pred_sentiment == label_humana_str)

            rows.append({
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'acierto': acierto
            })

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Comparar tasa de acierto por grupo ===
grupos = ['gender', 'ideologia', 'age_group', 'party']
resultados = []

for variable in grupos:
    grupos_unicos = df[variable].dropna().unique()
    if len(grupos_unicos) != 2:
        continue

    g1, g2 = grupos_unicos
    acc1 = df[df[variable] == g1]['acierto']
    acc2 = df[df[variable] == g2]['acierto']

    if len(acc1) < 10 or len(acc2) < 10:
        continue

    # Diferencia en tasa de acierto
    mean1, mean2 = acc1.mean(), acc2.mean()
    d = cohens_d(acc1, acc2)
    t_stat, p_value = ttest_ind(acc1, acc2, equal_var=False)

    resultados.append({
        'grupo': variable,
        'grupo_1': g1,
        'grupo_2': g2,
        'acierto_1': mean1,
        'acierto_2': mean2,
        'diff': mean1 - mean2,
        'd_cohen': d,
        't_stat': t_stat,
        'p_value': p_value,
        'n1': len(acc1),
        'n2': len(acc2)
    })

# === 6. Resultados ordenados por magnitud de efecto ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)

# === 7. Guardar como CSV ===
df_resultados.to_csv("sesgo_modelo_sentimiento.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'sesgo_modelo_sentimiento.csv'")


In [4]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === 1. Cargar archivo JSON con estructura por nombre de político ===
with open('ingleses/fusion_final_con_odio_y_stanza.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer datos de tweets y replies con métricas y atributos del autor ===
rows = []
for nombre, info in data.items():
    gender = info.get('Genero')
    age = info.get('Edad')
    party = info.get('mp_party')
    ideologia = info.get('Izquierda/Derecha')
    etnia = info.get('Etnia')

    for tweet in info.get('tweets', []):
        fuentes = [tweet] + tweet.get('replies', [])

        for fuente in fuentes:
            if not isinstance(fuente, dict):
                continue

            row = {
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'etnia': etnia,
                'hate_score': fuente.get('hate_score'),
                'sentiment_score': fuente.get('sentiment_score'),
            }

            # Añadir emociones específicas si están presentes
            emotion_label = fuente.get('emotion_label')
            emotion_score = fuente.get('emotion_score')

            if emotion_label in ['joy', 'anger', 'sadness', 'optimism']:
                row[emotion_label] = emotion_score

            rows.append(row)

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Función para calcular d de Cohen ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled

# === 6. Comparar métricas entre grupos ===
metricas = ['hate_score', 'sentiment_score', 'anger', 'joy', 'sadness', 'optimism']
grupos = ['gender', 'ideologia', 'age_group']

resultados = []

for variable_grupo in grupos:
    for metrica in metricas:
        grupos_unicos = df[variable_grupo].dropna().unique()
        if len(grupos_unicos) != 2:
            continue

        grupo1, grupo2 = grupos_unicos
        datos1 = df[df[variable_grupo] == grupo1][metrica].dropna()
        datos2 = df[df[variable_grupo] == grupo2][metrica].dropna()

        if len(datos1) < 10 or len(datos2) < 10:
            continue

        d = cohens_d(datos1, datos2)
        t_stat, p_value = ttest_ind(datos1, datos2, equal_var=False)

        resultados.append({
            "grupo": variable_grupo,
            "valor": metrica,
            "grupo_1": grupo1,
            "grupo_2": grupo2,
            "d_cohen": d,
            "t_stat": t_stat,
            "p_value": p_value,
            "n1": len(datos1),
            "n2": len(datos2)
        })

# === 7. Guardar resultados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)
df_resultados.to_csv("comparaciones_britanicos.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'comparaciones_britanicos.csv'")


✅ Resultados guardados en 'comparaciones_britanicos.csv'


In [14]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === Función para calcular el tamaño del efecto (Cohen's d) ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0.0

# === 1. Cargar archivo JSON ===
with open('spain/Completado/Spain_Completo.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer predicciones y etiquetas humanas ===
rows = []
for usuario in data:
    gender = usuario.get('gender')
    age = usuario.get('age')
    party = usuario.get('party')
    ideologia = usuario.get('ideologia')  # si está disponible

    for tweet in usuario.get('tweets', []):
        # Elegimos tweet original + replies
        for fuente in [tweet] + tweet.get('replies', []):
            if not isinstance(fuente, dict):
                continue

            # Etiqueta humana de sentimiento
            label_humana = fuente.get('annotations', {}).get('Total')
            if label_humana not in [-1, 0, 1, -1.0, 0.0, 1.0]:
                continue

            # Etiqueta del modelo
            analysis = fuente.get('analysis')
            if not isinstance(analysis, dict):
                continue

            pred_sentiment = analysis.get('sentiment', {}).get('label')
            if pred_sentiment not in ['positive', 'neutral', 'negative']:
                continue

            # Convertimos la etiqueta humana a texto para comparar
            mapping = {-1: 'negative', 0: 'neutral', 1: 'positive'}
            label_humana_str = mapping.get(int(label_humana))

            acierto = int(pred_sentiment == label_humana_str)

            rows.append({
                'gender': gender,
                'age': age,
                'party': party,
                'ideologia': ideologia,
                'acierto': acierto
            })

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Comparar tasa de acierto por grupo ===
grupos = ['gender', 'ideologia', 'age_group', 'party']
resultados = []

for variable in grupos:
    grupos_unicos = df[variable].dropna().unique()
    if len(grupos_unicos) != 2:
        continue

    g1, g2 = grupos_unicos
    acc1 = df[df[variable] == g1]['acierto']
    acc2 = df[df[variable] == g2]['acierto']

    if len(acc1) < 10 or len(acc2) < 10:
        continue

    mean1, mean2 = acc1.mean(), acc2.mean()
    d = cohens_d(acc1, acc2)
    t_stat, p_value = ttest_ind(acc1, acc2, equal_var=False)

    resultados.append({
        'grupo': variable,
        'grupo_1': g1,
        'grupo_2': g2,
        'acierto_1': mean1,
        'acierto_2': mean2,
        'diff': mean1 - mean2,
        'd_cohen': d,
        't_stat': t_stat,
        'p_value': p_value,
        'n1': len(acc1),
        'n2': len(acc2)
    })

# === 6. Resultados ordenados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)

# === 7. Guardar resultados ===
df_resultados.to_csv("sesgo_modelo_sentimiento_espanoles.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'sesgo_modelo_sentimiento_espanoles.csv'")


✅ Resultados guardados en 'sesgo_modelo_sentimiento_espanoles.csv'


In [18]:
import json
import pandas as pd

# === 1. Cargar archivo JSON ===
with open("spain/Completado/Spain_Completo.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === 2. Extraer datos de cada político ===
rows = []
for politico in data:
    gender = politico.get("gender")
    age = politico.get("age")
    party = politico.get("party")
    ideologia = politico.get("ideologia")
    rows.append({
        "gender": gender,
        "age": age,
        "party": party,
        "ideologia": ideologia
    })

df = pd.DataFrame(rows)

# === 3. Crear resumen ===
resumen = {
    "Total de políticos": [len(df)],
    "Número de hombres": [df['gender'].str.lower().eq("masculino").sum()],
    "Número de mujeres": [df['gender'].str.lower().eq("femenino").sum()]
}
df_resumen = pd.DataFrame(resumen)

# === 4. Distribución por partido e ideología ===
dist_partido = df['party'].value_counts().rename_axis('partido').reset_index(name='n_politicos')
dist_ideologia = df['ideologia'].value_counts().rename_axis('ideologia').reset_index(name='n_politicos')

# === 5. Guardar todo en un archivo Excel con varias hojas ===
with pd.ExcelWriter("resumen_politicos_espanoles.xlsx") as writer:
    df_resumen.to_excel(writer, sheet_name="Resumen General", index=False)
    dist_partido.to_excel(writer, sheet_name="Por Partido", index=False)
    dist_ideologia.to_excel(writer, sheet_name="Por Ideología", index=False)

print("✅ Excel generado como 'resumen_politicos_espanoles.xlsx'")


✅ Excel generado como 'resumen_politicos_espanoles.xlsx'


In [17]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# === Función para calcular el tamaño del efecto (Cohen's d) ===
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0.0

# === 1. Cargar archivo JSON ===
with open('ingleses/Ingleses_Completo.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# === 2. Extraer predicciones comparadas con etiqueta humana ===
rows = []
for tweet in data:
    mp = tweet.get("mp", {})
    gender = mp.get("genero")
    age = mp.get("edad")
    party = mp.get("party")
    ideologia = mp.get("ideologia")
    pais = mp.get("pais")
    etnia = mp.get("etnia")

    acierto = tweet.get("match_sentiment_label")
    if isinstance(acierto, bool):
        rows.append({
            'gender': gender,
            'age': age,
            'party': party,
            'ideologia': ideologia,
            'pais': pais,
            'etnia': etnia,
            'acierto': int(acierto)  # convertir a 1 (True) o 0 (False)
        })

# === 3. Crear DataFrame ===
df = pd.DataFrame(rows)

# === 4. Categorizar edad ===
df = df.dropna(subset=['age'])
df['age_group'] = pd.cut(df['age'], bins=[0, 40, 100], labels=['joven', 'mayor'])

# === 5. Comparar tasa de acierto por grupo ===
grupos = ['gender', 'ideologia', 'age_group', 'party', 'etnia']
resultados = []

for variable in grupos:
    grupos_unicos = df[variable].dropna().unique()
    if len(grupos_unicos) != 2:
        continue  # solo comparar si hay dos grupos

    g1, g2 = grupos_unicos
    acc1 = df[df[variable] == g1]['acierto']
    acc2 = df[df[variable] == g2]['acierto']

    if len(acc1) < 10 or len(acc2) < 10:
        continue

    mean1, mean2 = acc1.mean(), acc2.mean()
    d = cohens_d(acc1, acc2)
    t_stat, p_value = ttest_ind(acc1, acc2, equal_var=False)

    resultados.append({
        'grupo': variable,
        'grupo_1': g1,
        'grupo_2': g2,
        'acierto_1': mean1,
        'acierto_2': mean2,
        'diff': mean1 - mean2,
        'd_cohen': d,
        't_stat': t_stat,
        'p_value': p_value,
        'n1': len(acc1),
        'n2': len(acc2)
    })

# === 6. Resultados ordenados ===
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='d_cohen', key=lambda x: abs(x), ascending=False)

# === 7. Guardar resultados ===
df_resultados.to_csv("sesgo_modelo_sentimiento_estructura_nueva.csv", index=False, encoding='utf-8')
print("✅ Resultados guardados en 'sesgo_modelo_sentimiento_estructura_nueva.csv'")


✅ Resultados guardados en 'sesgo_modelo_sentimiento_estructura_nueva.csv'


In [14]:
!pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import json
import pandas as pd

# Cargar datos
with open('tweets_fusionados_ideologia.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Recolectar datos de tweets originales con anotación
rows = []
for usuario in data:
    gender = usuario['gender']
    age = usuario['age']
    party = usuario['party']
    ideologia = usuario['ideologia']
    age_group = 'joven' if age is not None and age <= 40 else 'mayor' if age is not None else None

    for tweet in usuario['tweets']:
        annotations = tweet.get('annotations', {})
        total = annotations.get('Total')
        if total is None:
            continue

        # IA 1: Cardiff (sentiment.label)
        sentiment_label = tweet['analysis'].get('sentiment', {}).get('label')

        # IA 2: Stanza (int: -1, 0, 1)
        sentiment_stanza = tweet['analysis'].get('sentiment_stanza')

        # Convertir label de texto a número
        label_map = {'negative': -1, 'neutral': 0, 'positive': 1}
        sentiment_cardiff = label_map.get(sentiment_label)

        row = {
            "gender": gender,
            "age_group": age_group,
            "ideologia": ideologia,
            "Total": total,
            "Cardiff": sentiment_cardiff,
            "Stanza": sentiment_stanza
        }
        rows.append(row)

# Crear DataFrame
df = pd.DataFrame(rows)

# Comparar predicciones con etiqueta humana
df['acierto_cardiff'] = df['Cardiff'] == df['Total']
df['acierto_stanza'] = df['Stanza'] == df['Total']

# Agrupar por atributo y calcular tasa de acierto
def tasas_acierto_por(variable):
    resumen = df.groupby(variable)[['acierto_cardiff', 'acierto_stanza']].mean() * 100
    resumen = resumen.rename(columns={
        'acierto_cardiff': 'Cardiff (%)',
        'acierto_stanza': 'Stanza (%)'
    })
    return resumen.round(2)

# Calcular tasas
tasas_por_genero = tasas_acierto_por('gender')
tasas_por_ideologia = tasas_acierto_por('ideologia')
tasas_por_edad = tasas_acierto_por('age_group')

# Guardar a CSV
tasas_por_genero.to_csv('tasas_acierto_genero.csv')
tasas_por_ideologia.to_csv('tasas_acierto_ideologia.csv')
tasas_por_edad.to_csv('tasas_acierto_edad.csv')

print("✅ Resultados guardados como:")
print("- tasas_acierto_genero.csv")
print("- tasas_acierto_ideologia.csv")
print("- tasas_acierto_edad.csv")


✅ Resultados guardados como:
- tasas_acierto_genero.csv
- tasas_acierto_ideologia.csv
- tasas_acierto_edad.csv


In [5]:
import json
from collections import defaultdict

# === 1. Cargar los archivos ===
with open("tweets_1_giregos.json", "r", encoding="utf-8") as f:
    tweets_data = json.load(f)

with open("bloque_1_20_politicos.json", "r", encoding="utf-8") as f:
    usuarios_extra = json.load(f)

# === 2. Crear índice auxiliar con búsqueda flexible ===
def buscar_info_extra(nombre):
    for entry in usuarios_extra:
        if nombre in entry['name']:
            return entry
    return {}

# === 3. Agrupar tweets por usuario ===
agrupados = defaultdict(lambda: {
    "name": None,
    "age": None,
    "gender": None,
    "party": None,
    "tweets": []
})

for tweet in tweets_data:
    nombre = tweet.get('name')
    partido = tweet.get('party')
    clave = (nombre, partido)

    # Buscar datos extra si faltan
    extra_info = buscar_info_extra(nombre)

    age = tweet.get("age") or extra_info.get("age")
    gender = tweet.get("gender") or extra_info.get("gender")
    party = tweet.get("party") or extra_info.get("party")

    # Asignar datos del usuario
    usuario = agrupados[clave]
    usuario["name"] = nombre
    usuario["age"] = age if age is not None else None
    usuario["gender"] = gender if gender is not None else None
    usuario["party"] = party if party is not None else None

    # Agregar tweet con toda su estructura
    tweet_info = {
        "id": tweet.get("id"),
        "user": tweet.get("user"),
        "tweet": tweet.get("tweet"),
        "label": tweet.get("label"),
        "annotator": tweet.get("annotator"),
        "analysis": tweet.get("analysis"),
        "replies": tweet.get("replies")
    }

    usuario["tweets"].append(tweet_info)

# === 4. Convertir a lista final y guardar ===
resultado = list(agrupados.values())

with open("tweets_agrupados_completos.json", "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print("✅ Tweets agrupados y completados con datos auxiliares. Guardado como 'tweets_agrupados_completos.json'")


✅ Tweets agrupados y completados con datos auxiliares. Guardado como 'tweets_agrupados_completos.json'


In [13]:
import json

# Cargar archivo
with open("grecia_anotados/Preprado/tweets_agrupados_completos_final.json", "r", encoding="utf-8") as f:
    usuarios = json.load(f)

# Contar totales
total_usuarios = len(usuarios)
total_tweets = sum(len(u.get("tweets", [])) for u in usuarios)
total_replies = sum(len(t.get("replies", [])) for u in usuarios for t in u.get("tweets", []))

# Mostrar resultados
print("📊 TOTALES GENERALES:")
print(f"- Total de usuarios: {total_usuarios}")
print(f"- Total de tweets: {total_tweets}")
print(f"- Total de replies: {total_replies}")


📊 TOTALES GENERALES:
- Total de usuarios: 168
- Total de tweets: 959
- Total de replies: 2328


In [7]:
import json

# Cargar archivo original
with open("tweets_agrupados_completos.json", "r", encoding="utf-8") as f:
    usuarios = json.load(f)

# Filtrar usuarios válidos
usuarios_limpios = [
    u for u in usuarios
    if not (isinstance(u.get("name"), str) and u["name"].startswith("Error al extraer usuario"))
]

# Guardar archivo limpio
with open("tweets_agrupados_sin_errores.json", "w", encoding="utf-8") as f:
    json.dump(usuarios_limpios, f, indent=2, ensure_ascii=False)

print(f"✅ Eliminados {len(usuarios) - len(usuarios_limpios)} usuarios con nombre erróneo.")
print("📁 Archivo limpio guardado como 'tweets_agrupados_sin_errores.json'")


✅ Eliminados 41 usuarios con nombre erróneo.
📁 Archivo limpio guardado como 'tweets_agrupados_sin_errores.json'


In [11]:
import json
import pandas as pd

# === 1. Cargar usuarios con metadatos corregidos desde Excel ===
df_corr = pd.read_excel("usuarios_incompletos_completados_updated.xlsx")

# Normalizar los nombres para buscar por inclusión parcial
usuarios_corr = []
for _, row in df_corr.iterrows():
    usuarios_corr.append({
        "name": str(row["name"]).strip().lower(),
        "age": row.get("age"),
        "gender": row.get("gender"),
        "party": row.get("party")
    })

# === 2. Cargar el archivo JSON original de tweets agrupados ===
with open("tweets_agrupados_sin_errores.json", "r", encoding="utf-8") as f:
    datos = json.load(f)

# === 3. Función para buscar en lista de usuarios corregidos ===
def buscar_metadata(nombre):
    nombre = nombre.strip().lower()
    for u in usuarios_corr:
        if nombre in u["name"] or u["name"] in nombre:
            return u
    return {}

# === 4. Completar los campos que estén a None ===
for usuario in datos:
    if usuario.get("age") is None or usuario.get("gender") is None or usuario.get("party") is None:
        info = buscar_metadata(usuario["name"])
        if usuario.get("age") is None:
            usuario["age"] = info.get("age")
        if usuario.get("gender") is None:
            usuario["gender"] = info.get("gender")
        if usuario.get("party") is None:
            usuario["party"] = info.get("party")

# === 5. Guardar el nuevo JSON completo ===
with open("tweets_agrupados_completos_final.json", "w", encoding="utf-8") as f:
    json.dump(datos, f, indent=2, ensure_ascii=False)

print("✅ Archivo actualizado guardado como 'tweets_agrupados_completos_final.json'")


✅ Archivo actualizado guardado como 'tweets_agrupados_completos_final.json'


In [ ]:
import json
import pandas as pd

# Cargar el archivo limpio y agrupado
with open('tweets_agrupados_sin_errores.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Recolectar datos de tweets originales con anotación
rows = []
for usuario in data:
    gender = usuario.get('gender')
    age = usuario.get('age')
    party = usuario.get('party')
    age_group = 'joven' if age is not None and age <= 40 else 'mayor' if age is not None else None

    for tweet in usuario.get('tweets', []):
        annotations = tweet.get('annotations', {})
        total = annotations.get('Total')
        if total is None:
            continue

        # IA 1: Cardiff (sentiment.label)
        sentiment_label = tweet['analysis'].get('sentiment', {}).get('label')

        # IA 2: Stanza (int: -1, 0, 1)

        # Convertir label de texto a número
        label_map = {'negative': -1, 'neutral': 0, 'positive': 1}
        sentiment_cardiff = label_map.get(sentiment_label)

        row = {
            "gender": gender,
            "age_group": age_group,
            "party": party,
            "Total": total,
            "Cardiff": sentiment_cardiff,
        }
        rows.append(row)

# Crear DataFrame
df = pd.DataFrame(rows)

# Comparar predicciones con etiqueta humana
df['acierto_cardiff'] = df['Cardiff'] == df['Total']

# Agrupar por atributo y calcular tasa de acierto
def tasas_acierto_por(variable):
    resumen = df.groupby(variable)[['acierto_cardiff', 'acierto_stanza']].mean() * 100
    resumen = resumen.rename(columns={
        'acierto_cardiff': 'Cardiff (%)',
        'acierto_stanza': 'Stanza (%)'
    })
    return resumen.round(2)

# Calcular tasas
tasas_por_genero = tasas_acierto_por('gender')
tasas_por_partido = tasas_acierto_por('party')
tasas_por_edad = tasas_acierto_por('age_group')

# Guardar a CSV
tasas_por_genero.to_csv('tasas_acierto_genero.csv')
tasas_por_partido.to_csv('tasas_acierto_partido.csv')
tasas_por_edad.to_csv('tasas_acierto_edad.csv')

print("✅ Resultados guardados como:")
print("- tasas_acierto_genero.csv")
print("- tasas_acierto_partido.csv")
print("- tasas_acierto_edad.csv")


In [2]:
import json
import stanza

# Carga el pipeline en español
nlp = stanza.Pipeline(lang='es', processors='tokenize,pos', tokenize_pretokenized=False)

# Lista de etiquetas POS irrelevantes que queremos eliminar
irrelevant_pos = {"DET", "ADP", "PUNCT", "CCONJ", "SCONJ", "PART"}

def get_filtered_pos(text):
    doc = nlp(text)
    pos_tags = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos not in irrelevant_pos:
                pos_tags.append(word.upos)
    return pos_tags

# Carga tus datos (por ejemplo desde un archivo JSON)
with open("spain/Completado/Spain_Completo.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Procesa cada tweet y sus replies
for person in data:
    for tweet in person["tweets"]:
        tweet["filtered_pos_tags"] = get_filtered_pos(tweet["tweet"])
        for reply in tweet.get("replies", []):
            reply["filtered_pos_tags"] = get_filtered_pos(reply["reply"])

# Guarda el archivo con las nuevas etiquetas POS filtradas
with open("datos_filtrados.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


2025-05-06 10:47:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-06 10:47:18 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-05-06 10:47:18 WARNING: Language es package default expects mwt, which has been added
2025-05-06 10:47:19 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2025-05-06 10:47:19 INFO: Using device: cuda
2025-05-06 10:47:19 INFO: Loading: tokenize
2025-05-06 10:47:19 INFO: Loading: mwt
2025-05-06 10:47:19 INFO: Loading: pos
2025-05-06 10:47:20 INFO: Done loading processors!


In [3]:
import stanza
import json

# Configura el pipeline de inglés
stanza.download("en")
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos', tokenize_pretokenized=False)

# Etiquetas POS que vamos a filtrar (irrelevantes para análisis semántico)
irrelevant_pos = {"DET", "ADP", "PUNCT", "CCONJ", "SCONJ", "PART", "SYM", "X"}

def get_filtered_pos(text):
    doc = nlp(text)
    pos_tags = []
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos not in irrelevant_pos:
                pos_tags.append(word.upos)
    return pos_tags

# Cargar los datos
with open("ingleses/Ingleses_Completo.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Procesar cada entrada del JSON
for tweet in data:
    tweet["filtered_pos_tags"] = get_filtered_pos(tweet["text"])

# Guardar los resultados en un nuevo archivo
with open("tweets_filtrados_ingleses.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)


2025-05-06 11:09:00 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-05-06 11:09:00 INFO: Downloading default packages for language: en (English) ...
2025-05-06 11:09:01 INFO: File exists: /home/jupyter-lquijano/stanza_resources/en/default.zip
2025-05-06 11:09:04 INFO: Finished downloading models and saved to /home/jupyter-lquijano/stanza_resources
2025-05-06 11:09:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-06 11:09:04 INFO: Downloaded file to /home/jupyter-lquijano/stanza_resources/resources.json
2025-05-06 11:09:04 WARNING: Language en package default expects mwt, which has been added
2025-05-06 11:09:05 INFO: Loading these models for language: en (English):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2025-05-06 11:09:05 INFO: Using device: cuda
2025-05-06 11:09:05 INFO: Loading: tokenize
2025-05-06 11:09:05 INFO: Loading: mwt
2025-05-06 11:09:05 INFO: Loading: pos
2025-05-06 11:09:06 INFO: Done loading processors!


In [7]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from collections import defaultdict

# Cargar datos
with open("tweets_filtrados_ingleses.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Clasificación de edad
def categorizar_edad(edad):
    if edad < 40:
        return "joven"
    elif edad <= 60:
        return "adulto"
    else:
        return "mayor"

# Extraer filas con métricas útiles
rows = []
for entry in data:
    mp = entry["mp"]
    pos_tags = entry.get("filtered_pos_tags", [])
    total = len(pos_tags)
    if total == 0:
        continue

    row = {
        "pais": mp.get("pais", "Unknown"),
        "genero": mp.get("genero", "Unknown"),
        "etnia": mp.get("etnia", "Unknown"),
        "ideologia": mp.get("ideologia", "Unknown"),
        "party": mp.get("party", "Unknown"),
        "edad_grupo": categorizar_edad(mp.get("edad", 0)),
    }

    # POS tags
    tag_counts = defaultdict(int)
    for tag in pos_tags:
        tag_counts[tag] += 1
    for tag, count in tag_counts.items():
        row[f"POS_{tag}"] = count / total  # proporción

    # Emociones y scores
    nlp = entry.get("nlp", {})
    for key in ["sentiment_score", "hate_score", "emotion_score"]:
        val = nlp.get(key)
        if val is not None:
            row[key] = val
    for key in ["joy", "anger", "sadness", "optimism"]:
        if f"{key}_score" in nlp:
            row[key] = nlp[f"{key}_score"]

    rows.append(row)

# Crear DataFrame
df = pd.DataFrame(rows).fillna(0)

# Comparaciones a hacer
comparisons = {
    "genero": ["Male", "Female"],
    "ideologia": ["derecha", "izquierda"],
    "edad_grupo": ["joven", "mayor"]
}

# Variables a comparar
metricas = [col for col in df.columns if col.startswith("POS_")] + [
    "joy", "anger", "sadness", "optimism", "sentiment_score", "hate_score"
]

# Función de Cohen's d
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    if s_pooled == 0:
        return 0.0
    return (np.mean(x1) - np.mean(x2)) / s_pooled

# Analizar y guardar resultados
resultados = []

for grupo, (g1, g2) in comparisons.items():
    df1 = df[df[grupo] == g1]
    df2 = df[df[grupo] == g2]

    for metrica in metricas:
        if metrica not in df.columns:
            continue

        x1 = df1[metrica]
        x2 = df2[metrica]

        if len(x1) < 10 or len(x2) < 10:
            continue

        d = cohens_d(x1, x2)
        t_stat, p_value = ttest_ind(x1, x2, equal_var=False)

        resultados.append({
            "grupo": grupo,
            "valor": metrica,
            "grupo_1": g1,
            "grupo_2": g2,
            "d_cohen": d,
            "t_stat": t_stat,
            "p_value": p_value,
            "n1": len(x1),
            "n2": len(x2)
        })

# Convertir a DataFrame y guardar
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("cohens_d_analisis.csv", index=False)

print("✅ Resultados guardados en 'cohens_d_analisis.csv'")


✅ Resultados guardados en 'cohens_d_analisis.csv'


In [9]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from collections import defaultdict

# Cargar datos
with open("datos_filtrados.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Clasificación de edad
def categorizar_edad(edad):
    if edad < 40:
        return "joven"
    elif edad <= 60:
        return "adulto"
    else:
        return "mayor"

# Extraer datos tweet y replies
rows = []

for persona in data:
    genero = persona.get("gender", "Unknown")
    edad = persona.get("age", None)
    edad_grupo = categorizar_edad(edad) if edad is not None else "Unknown"
    party = persona.get("party", "Unknown")
    nombre = persona.get("name", "Unknown")
    pais = "esp"

    for tweet in persona.get("tweets", []):
        # Añadir tweet principal
        for fuente, pos_tags in [("tweet", tweet.get("filtered_pos_tags", []))] + \
                                 [("reply", reply.get("filtered_pos_tags", [])) for reply in tweet.get("replies", [])]:
            total = len(pos_tags)
            if total == 0:
                continue

            tag_counts = defaultdict(int)
            for tag in pos_tags:
                tag_counts[tag] += 1

            row = {
                "pais": pais,
                "genero": genero,
                "edad_grupo": edad_grupo,
                "party": party,
                "origen": fuente,
                "nombre": nombre
            }

            for tag, count in tag_counts.items():
                row[f"POS_{tag}"] = count / total

            rows.append(row)

# Crear DataFrame
df = pd.DataFrame(rows).fillna(0)

# Definir comparaciones posibles
comparisons = {}

# Género (si hay suficientes ejemplos)
generos = df["genero"].value_counts()
if len(generos) >= 2 and all(generos >= 10):
    comparisons["genero"] = list(generos.index[:2])

# Edad (joven vs mayor)
comparisons["edad_grupo"] = ["joven", "mayor"]

# Partido (si hay al menos 2 grupos grandes)
partidos = df["party"].value_counts()
if len(partidos) >= 2 and all(partidos[:2] >= 10):
    comparisons["party"] = list(partidos.index[:2])

# Variables a comparar (solo POS)
metricas = [col for col in df.columns if col.startswith("POS_")]

# Función de Cohen's d
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    if s_pooled == 0:
        return 0.0
    return (np.mean(x1) - np.mean(x2)) / s_pooled

# Ejecutar comparaciones
resultados = []

for grupo, (g1, g2) in comparisons.items():
    df1 = df[df[grupo] == g1]
    df2 = df[df[grupo] == g2]

    for metrica in metricas:
        x1 = df1[metrica]
        x2 = df2[metrica]

        if len(x1) < 10 or len(x2) < 10:
            continue

        d = cohens_d(x1, x2)
        t_stat, p_value = ttest_ind(x1, x2, equal_var=False)

        resultados.append({
            "grupo": grupo,
            "valor": metrica,
            "grupo_1": g1,
            "grupo_2": g2,
            "d_cohen": round(d, 5),
            "t_stat": round(t_stat, 5),
            "p_value": p_value,
            "n1": len(x1),
            "n2": len(x2)
        })

# Exportar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("cohens_d_espanol.csv", index=False)

print("✅ Análisis completo. Resultados guardados en 'cohens_d_espanol.csv'")


✅ Análisis completo. Resultados guardados en 'cohens_d_espanol.csv'


In [10]:
import json
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Cargar datos griegos
with open("grecia_anotados/Preprado/Griegos_f.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Clasificar edad
def categorizar_edad(edad):
    if edad < 40:
        return "joven"
    elif edad <= 60:
        return "adulto"
    else:
        return "mayor"

# Extraer tweets y replies con métricas POS
rows = []

for persona in data:
    genero = persona.get("gender", "Unknown")
    edad = persona.get("age", None)
    edad_grupo = categorizar_edad(edad) if edad is not None else "Unknown"
    partido = persona.get("party", "Unknown")
    ideologia = persona.get("polaridad_ideologica", "Unknown")
    nombre = persona.get("name", "Unknown")
    pais = "gre"

    for tweet in persona.get("tweets", []):
        analysis = tweet.get("analysis", {})
        pos = analysis.get("pos_features", {})
        if not pos:
            continue

        row = {
            "pais": pais,
            "nombre": nombre,
            "gender": genero,
            "edad_grupo": edad_grupo,
            "party": partido,
            "ideologia": ideologia,
        }

        for k, v in pos.items():
            # convertir a número si es posible
            if isinstance(v, (int, float)) or (isinstance(v, str) and v.replace('.', '', 1).isdigit()):
                row[k] = float(v)

        rows.append(row)

# Crear DataFrame
df = pd.DataFrame(rows).fillna(0)

# Comparaciones posibles
comparisons = {}

if df["gender"].nunique() >= 2:
    top2 = df["gender"].value_counts().index[:2]
    comparisons["gender"] = list(top2)

if df["edad_grupo"].nunique() >= 2:
    comparisons["edad_grupo"] = ["joven", "mayor"]

if df["party"].nunique() >= 2:
    top2 = df["party"].value_counts().index[:2]
    comparisons["party"] = list(top2)

if df["ideologia"].nunique() >= 2:
    top2 = df["ideologia"].value_counts().index[:2]
    comparisons["ideologia"] = list(top2)

# Métricas POS
metricas = [col for col in df.columns if col.startswith("num_") or col.startswith("ratio_")]

# Función para Cohen's d
def cohens_d(x1, x2):
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s_pooled = np.sqrt(((n1 - 1)*s1 + (n2 - 1)*s2) / (n1 + n2 - 2))
    return (np.mean(x1) - np.mean(x2)) / s_pooled if s_pooled > 0 else 0

# Comparar y guardar resultados
resultados = []

for grupo, (g1, g2) in comparisons.items():
    df1 = df[df[grupo] == g1]
    df2 = df[df[grupo] == g2]

    for metrica in metricas:
        if metrica not in df.columns:
            continue

        x1 = df1[metrica]
        x2 = df2[metrica]

        if len(x1) < 10 or len(x2) < 10:
            continue

        d = cohens_d(x1, x2)
        t_stat, p_value = ttest_ind(x1, x2, equal_var=False)

        resultados.append({
            "grupo": grupo,
            "valor": metrica,
            "grupo_1": g1,
            "grupo_2": g2,
            "d_cohen": round(d, 5),
            "t_stat": round(t_stat, 5),
            "p_value": p_value,
            "n1": len(x1),
            "n2": len(x2)
        })

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("cohens_d_griegos.csv", index=False)

print("✅ Resultados guardados en 'cohens_d_griegos.csv'")


✅ Resultados guardados en 'cohens_d_griegos.csv'
